In [1]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from load_data_and_distribution_analisys import split_dataset_v1, load_dataset_cleaned

# Random forest con feature reduction e cross validation

In questa ultima parte relativa l'utilizzo della RandomForest come modello di appredimento, useremo la cross validation per valutare in maniera più precisa le performance dopo aver fatto feature selection.
Siccome nello step precedente abbiamo allenato il nostro modello tenendo però costante il set di train e di validazione, potremmo aver "overfittato" in base alla suddivisione specifica; con la cross-validation, testiamo il modello su diverse porzioni del dataset e possiamo valutare se le feature scelte migliorano davvero il modello.

In [ ]:
dataset_cleaned = load_dataset_cleaned()

In [ ]:
X_smoke, Y_smoke, X_drink, Y_drink = split_dataset_v1(dataset_cleaned)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_smoke, Y_smoke, test_size=0.3, random_state=42)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_drink, Y_drink, test_size=0.3, random_state=42)

In [ ]:
best_selected_features_d = ["sex", "age", "gamma_GTP", "HDL_chole", "SGOT_ALT", "weight", "SGOT_AST", "tot_chole", "triglyceride", "LDL_chole", "serum_creatinine", "DBP", "hemoglobin", "BLDS", "hear_right", "waistline", "hear_left", "height"]
best_selected_features_s = ["age", "sex", "height", "weight", "hear_right", "sight_right", "DBP", "BLDS", "HDL_chole", "LDL_chole", "triglyceride", "serum_creatinine", "SGOT_AST", "SGOT_ALT", "gamma_GTP", "hemoglobin"]

In [ ]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42, class_weight="balanced")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_score_rf_s = cross_val_score(forest, X_smoke.loc[:, best_selected_features_s], Y_smoke, cv=cv, scoring='f1_weighted', n_jobs=-1)
accuracy_rf_s = cross_val_score(forest, X_smoke.loc[:, best_selected_features_s], Y_smoke, cv=cv, scoring='balanced_accuracy', n_jobs=-1)
print(f"F1-score medio: {f1_score_rf_s.mean()*100:.2f}%")
print(f"Accuratezza media: {accuracy_rf_s.mean()*100:.2f}%")

F1-score medio: 70.42%
Accuratezza media: 65.02%


In [ ]:
forest = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_split=50, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_score_rf_d = cross_val_score(forest, X_drink.loc[:, best_selected_features_d], Y_drink, cv=cv, scoring='f1', n_jobs=-1)
accuracy_rf_d = cross_val_score(forest, X_drink.loc[:, best_selected_features_d], Y_drink, cv=cv, scoring='accuracy', n_jobs=-1)
print(f"F1-score medio: {f1_score_rf_d.mean()*100:.2f}%")
print(f"Accuratezza media: {accuracy_rf_d.mean()*100:.2f}%")

F1-score medio: 73.56%
Accuratezza media: 73.11%
